In [2]:
# get all imports out of the way
import tensorneat as tn
import jax.numpy as jnp
import pandas as pd

In [13]:
# drop nans split by symbol and join on date
file_path = "hist_data/stocks/sp500_stocks.csv"  # Update this path if needed
df = pd.read_csv(file_path)

df = df.dropna()

symbols = df['Symbol'].unique()
symbol_dfs = {symbol: df[df['Symbol'] == symbol] for symbol in symbols}

merged_df = symbol_dfs[symbols[0]].set_index('Date')

for symbol in symbols[1:]:
    symbol_data = symbol_dfs[symbol].set_index('Date')
    if (len(symbol_data) == 3768):
        merged_df = merged_df.join(symbol_data, how='inner', rsuffix=f"_{symbol}")

tensorneat_input = merged_df.to_numpy()

print(tensorneat_input.shape)

(3768, 1050)


In [14]:
print(merged_df.head())

           Symbol  Adj Close     Close      High       Low      Open  \
Date                                                                   
2010-01-04    AOS   5.937266  7.435000  7.480000  7.261667  7.295000   
2010-01-05    AOS   5.861404  7.340000  7.431667  7.308333  7.431667   
2010-01-06    AOS   5.864068  7.343333  7.405000  7.301667  7.335000   
2010-01-07    AOS   5.881369  7.365000  7.425000  7.311667  7.356667   
2010-01-08    AOS   5.967879  7.473333  7.485000  7.311667  7.331667   

               Volume Symbol_ABT  Adj Close_ABT  Close_ABT  ...    Low_WMB  \
Date                                                        ...              
2010-01-04  1104600.0        ABT      18.763718  26.129908  ...  17.445280   
2010-01-05  1207200.0        ABT      18.612118  25.918797  ...  17.534952   
2010-01-06   663000.0        ABT      18.715481  26.062737  ...  17.771360   
2010-01-07   564000.0        ABT      18.870522  26.278646  ...  18.072985   
2010-01-08   504600.0      

In [ ]:
# create a target array

close_columns = [col for col in merged_df.columns if col.split('_')[0] == 'Close']
target = merged_df[close_columns].pct_change().shift(-1).values
print(target.shape)

(3768, 300)


In [16]:
feature_df = merged_df[close_columns]
for c in close_columns:
    feature_df[f'{c}_ma5'] = feature_df[c].rolling(window=5).mean()
    feature_df[f'{c}_ma10'] = feature_df[c].rolling(window=10).mean()
    feature_df[f'{c}_ma20'] = feature_df[c].rolling(window=20).mean()
    feature_df[f'{c}_avg_change_13'] = feature_df[c].pct_change().rolling(window=13).mean()
    feature_df[f'{c}_stddev_13'] = feature_df[c].pct_change().rolling(window=13).std()
print(feature_df.head())

C:\Users\nick5\AppData\Local\Temp\ipykernel_40800\2104017220.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_df[f'{c}_ma5'] = feature_df[c].rolling(window=5).mean()
C:\Users\nick5\AppData\Local\Temp\ipykernel_40800\2104017220.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_df[f'{c}_ma10'] = feature_df[c].rolling(window=10).mean()
C:\Users\nick5\AppData\Local\Temp\ipykernel_40800\2104017220.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

            Adj Close     Close  Adj Close_ABT  Close_ABT  Adj Close_AES  \
Date                                                                       
2010-01-04   5.937266  7.435000      18.763718  26.129908       9.370146   
2010-01-05   5.861404  7.340000      18.612118  25.918797       9.274181   
2010-01-06   5.864068  7.343333      18.715481  26.062737       9.178215   
2010-01-07   5.881369  7.365000      18.870522  26.278646       9.185070   
2010-01-08   5.967879  7.473333      18.966991  26.412991       9.445544   

            Close_AES  Adj Close_ALB  Close_ALB  Adj Close_ALL  Close_ALL  \
Date                                                                        
2010-01-04      13.67      30.864571  38.090000      21.624144  30.410000   
2010-01-05      13.53      30.807838  38.020000      21.979689  30.910000   
2010-01-06      13.39      30.864571  38.090000      21.986795  30.920000   
2010-01-07      13.40      30.645794  37.820000      22.100569  31.080000   
2010-

C:\Users\nick5\AppData\Local\Temp\ipykernel_40800\2104017220.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feature_df[f'{c}_ma5'] = feature_df[c].rolling(window=5).mean()
C:\Users\nick5\AppData\Local\Temp\ipykernel_40800\2104017220.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feature_df[f'{c}_ma10'] = feature_df[c].rolling(window=10).mean()
C:\Users\nick5\AppData\Local\Temp\ipykernel_40800\2104017220.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many 